In [11]:
import numpy as np, pandas as pd
from pathlib import Path
pd.set_option("display.float_format", "{:,.6f}".format)

OUT = Path.cwd().parents[1] / "outputs" / "01_CIR"
files = {
    "Baseline": OUT / "MC_Baseline_higher_for_longer.npy",
    "Tariff":   OUT / "MC_Tariff_escalation.npy",
    "Deesc":    OUT / "MC_Deescalation_growth_wobble.npy",
}

stats = []
for name, f in files.items():
    arr = np.load(f)     # shape (500, 504)
    stats.append({
        "scenario": name,
        "mean_rate": arr.mean(),
        "ann_vol": arr.std()*np.sqrt(252),
        "terminal_mean": arr[:,-1].mean(),
    })
pd.DataFrame(stats)

,scenario,mean_rate,ann_vol,terminal_mean
0,Baseline,0.044953,0.080480,0.039452
1,Tariff,0.046894,0.083719,0.042488
2,Deesc,0.040236,0.097323,0.032732


In [15]:
# Compute metrics from saved MC paths and write CSV
import numpy as np, pandas as pd
from pathlib import Path
import os

PROJECT_ROOT = Path.cwd().parents[1]
OUT_DIR = PROJECT_ROOT / "outputs" / "01_CIR"
SCENARIOS = {
    "Baseline_higher_for_longer": dict(theta=0.0325),
    "Tariff_escalation":          dict(theta=0.0350),
    "Deescalation_growth_wobble": dict(theta=0.0275),
}

THRESH_UP = 0.055   # 5.5%
THRESH_DN = 0.030   # 3.0%

def metrics_from_paths(paths: np.ndarray, theta: float,
                       thresh_up: float = THRESH_UP, thresh_dn: float = THRESH_DN):
    """
    paths: (n_paths, steps) of short-rate decimals
    return: dict with mean, p5, p95 across paths for each metric
    """
    r = np.asarray(paths)                             # (N,T)
    avg_rate = r.mean(axis=1)                         # (N,)
    ann_vol  = r.std(axis=1) * np.sqrt(252)
    pct_above = (r > thresh_up).mean(axis=1)
    pct_below = (r < thresh_dn).mean(axis=1)
    terminal  = r[:, -1]

    def row(stat, arr):
        return pd.Series({
            "avg_rate": arr.mean() if stat=="mean" else np.percentile(arr, 5 if stat=="p5" else 95),
            "ann_vol": arr.mean() if stat=="mean" else np.percentile(ann_vol, 5 if stat=="p5" else 95)
        })

    # Build full table
    df = pd.DataFrame({
        "avg_rate":    avg_rate,
        "ann_vol":     ann_vol,
        "pct_time>5.5%": pct_above,
        "pct_time<3.0%": pct_below,
        "terminal_rate": terminal
    })
    # Summaries
    summaries = []
    for stat, func in [("mean", np.mean), ("p5", lambda x: np.percentile(x,5)), ("p95", lambda x: np.percentile(x,95))]:
        summaries.append(pd.Series({
            "avg_rate":        func(df["avg_rate"]),
            "ann_vol":         func(df["ann_vol"]),
            "pct_time>5.5%":   func(df["pct_time>5.5%"]),
            "pct_time<3.0%":   func(df["pct_time<3.0%"]),
            "terminal_rate":   func(df["terminal_rate"]),
            "theta_used":      theta
        }, name=stat))
    return pd.DataFrame(summaries)

rows = []
for name, meta in SCENARIOS.items():
    path_file = OUT_DIR / f"MC_{name}.npy"
    paths = np.load(path_file)
    tbl = metrics_from_paths(paths, theta=meta["theta"])
    tbl.insert(0, "scenario", name)
    rows.append(tbl)

metrics_df = pd.concat(rows).reset_index(names="stat")
csv_path = OUT_DIR / "cir_metrics.csv"
metrics_df.to_csv(csv_path, index=False)
print("✔ Wrote metrics:", csv_path)
metrics_df

✔ Wrote metrics: /Users/katherinecohen/Documents/FixedIncomePortfolio/outputs/01_CIR/cir_metrics.csv


,stat,scenario,avg_rate,ann_vol,pct_time>5.5%,pct_time<3.0%,terminal_rate,theta_used
0,mean,Baseline_higher_for_longer,0.044953,0.067273,0.003516,0.001306,0.039452,0.032500
1,p5,Baseline_higher_for_longer,0.040622,0.037647,0.000000,0.000000,0.033294,0.032500
2,p95,Baseline_higher_for_longer,0.048928,0.100167,0.017857,0.000000,0.045757,0.032500
3,mean,Tariff_escalation,0.046894,0.060300,0.031921,0.002008,0.042488,0.035000
4,p5,Tariff_escalation,0.041110,0.030244,0.000000,0.000000,0.034155,0.035000
5,p95,Tariff_escalation,0.052277,0.101300,0.193353,0.000000,0.051278,0.035000
6,mean,Deescalation_growth_wobble,0.040236,0.093247,0.000000,0.013337,0.032732,0.027500
7,p5,Deescalation_growth_wobble,0.037521,0.072154,0.000000,0.000000,0.029139,0.027500
8,p95,Deescalation_growth_wobble,0.042712,0.114185,0.000000,0.093452,0.036426,0.027500


In [16]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[1]
metrics_df = pd.read_csv(PROJECT_ROOT / "outputs" / "01_CIR" / "cir_metrics.csv")

mean_tbl = metrics_df.query("stat == 'mean'").set_index("scenario")
fmt_pct = lambda x: f"{x*100:.2f}%"
fmt_rt  = lambda x: f"{x*100:.2f}%"

b = mean_tbl.loc["Baseline_higher_for_longer"]
t = mean_tbl.loc["Tariff_escalation"]
d = mean_tbl.loc["Deescalation_growth_wobble"]

memo = f"""# Rates Outlook — CIR (Tariff-Sensitive Scenarios)
**Date:** {pd.Timestamp.today().date()}  
**Analyst:** Katherine Cohen

## Key Metrics (Monte Carlo mean across 500 paths)
| Scenario | Avg rate | Ann vol | Time > 5.5% | Time < 3% | Terminal |
|---|---:|---:|---:|---:|---:|
| Baseline | {fmt_rt(b['avg_rate'])} | {fmt_rt(b['ann_vol'])} | {fmt_pct(b['pct_time>5.5%'])} | {fmt_pct(b['pct_time<3.0%'])} | {fmt_rt(b['terminal_rate'])} |
| Tariff escalation | {fmt_rt(t['avg_rate'])} | {fmt_rt(t['ann_vol'])} | {fmt_pct(t['pct_time>5.5%'])} | {fmt_pct(t['pct_time<3.0%'])} | {fmt_rt(t['terminal_rate'])} |
| De-escalation | {fmt_rt(d['avg_rate'])} | {fmt_rt(d['ann_vol'])} | {fmt_pct(d['pct_time>5.5%'])} | {fmt_pct(d['pct_time<3.0%'])} | {fmt_rt(d['terminal_rate'])} |

## Interpretation & Actions
- **Tariff escalation:** higher θ and σ → rates stay elevated longer with bigger swings. Keep duration underweight vs. benchmark (focus on 2–7y KRD), add inflation protection, and trim HY/EM beta; consider payer swaptions/CDX hedges.
- **Baseline:** still elevated but less volatile; keep duration light and curve risk modest (bear-flattening bias).
- **De-escalation:** faster normalization (higher k) — begin adding duration (intermediate sector → long end), upgrade credit quality, and scale into spreads on weakness.

*Method:* CIR, daily steps over 2y, 500 paths per scenario; metrics computed per path, then averaged. Feller holds for all parameter sets.
"""
print(memo)

# Rates Outlook — CIR (Tariff-Sensitive Scenarios)
**Date:** 2025-08-30  
**Analyst:** Katherine Cohen

## Key Metrics (Monte Carlo mean across 500 paths)
| Scenario | Avg rate | Ann vol | Time > 5.5% | Time < 3% | Terminal |
|---|---:|---:|---:|---:|---:|
| Baseline | 4.50% | 6.73% | 0.35% | 0.13% | 3.95% |
| Tariff escalation | 4.69% | 6.03% | 3.19% | 0.20% | 4.25% |
| De-escalation | 4.02% | 9.32% | 0.00% | 1.33% | 3.27% |

## Interpretation & Actions
- **Tariff escalation:** higher θ and σ → rates stay elevated longer with bigger swings. Keep **duration underweight** vs. benchmark (focus on 2–7y KRD), add **inflation protection**, and **trim HY/EM beta**; consider payer swaptions/CDX hedges.
- **Baseline:** still elevated but less volatile; keep duration light and curve risk modest (bear-flattening bias).
- **De-escalation:** faster normalization (higher k) — begin **adding duration** (belly→long end), upgrade credit quality, and scale into spreads on weakness.

*Method:* CIR, dail